In [7]:
import requests
import xmltodict

base_url = "https://boardgamegeek.com/xmlapi2/thing"
params = {
    "id": 1,
    "type": "boardgame"
}

response = requests.get(url=base_url, params=params)

In [8]:
response_data = xmltodict.parse(response.content)

In [13]:
response_data

{'items': {'@termsofuse': 'https://boardgamegeek.com/xmlapi/termsofuse',
  'item': {'@type': 'boardgame',
   '@id': '1',
   'thumbnail': 'https://cf.geekdo-images.com/rpwCZAjYLD940NWwP3SRoA__thumb/img/YT6svCVsWqLrDitcMEtyazVktbQ=/fit-in/200x150/filters:strip_icc()/pic4718279.jpg',
   'image': 'https://cf.geekdo-images.com/rpwCZAjYLD940NWwP3SRoA__original/img/yR0aoBVKNrAmmCuBeSzQnMflLYg=/0x0/filters:format(jpeg)/pic4718279.jpg',
   'name': [{'@type': 'primary', '@sortindex': '5', '@value': 'Die Macher'},
    {'@type': 'alternate', '@sortindex': '1', '@value': '德国大选'},
    {'@type': 'alternate', '@sortindex': '1', '@value': '디 마허'}],
   'description': 'Die Macher is a game about seven sequential political races in different regions of Germany. Players are in charge of national political parties, and must manage limited resources to help their party to victory. The winning party will have the most victory points after all the regional elections. There are four different ways of scoring vi

In [11]:
a = prepare_data(response_data)

In [12]:
a

game_name,description,publication_year,min_players,max_players,best_num_players,recommended_num_players,suggested_play_age,categories,mechanics,families,designers,artists,publishers,playing_time,min_playtime,max_playtime,min_age,language_dependence_level,language_dependence_description
str,str,i64,i64,i64,str,str,str,list[str],list[str],list[str],list[str],list[str],list[str],i64,i64,i64,i64,str,str
"""Die Macher""","""Die Macher is a game about sev…",1986,3,5,"""5""","""4""","""14""","[""Economic"", ""Negotiation"", ""Political""]","[""Alliances"", ""Area Majority / Influence"", … ""Simultaneous Action Selection""]","[""Country: Germany"", ""Digital Implementations: VASSAL"", … ""Series: Classic Line (Valley Games)""]","[""Karl-Heinz Schmiel""]","[""Bernd Brunnhofer"", ""Marcus Gschwendtner"", ""Harald Lieske""]","[""Hans im Glück"", ""Moskito Spiele"", … ""YOKA Games""]",240,240,240,14,"""1""","""No necessary in-game text"""


In [1]:
import requests
import xmltodict
import asyncio
from typing import Dict, Tuple, Union, Optional, List

def make_request(url: str, params: dict) -> requests.Response:
    response = requests.get(url=url, params=params)
    return response

async def api_request(url: str, params: dict) -> requests.Response:
    return await asyncio.to_thread(make_request, url, params)

async def main():
    client = BGGClient()
    tasks = [client.get_game_market_data(i) for i in range(1, 10)]
    result = await asyncio.gather(*tasks)
    return result

class BGGClient:
    def __init__(self, base_url: str = "https://boardgamegeek.com/xmlapi2"):
        self.base_url = base_url

    async def get_game_market_data(self, game_id: int, retry_delay: int = 5, max_retries: int = 3) -> Tuple[str, Dict[str, Union[str, float]], float]:
        """
        Fetches detailed market information for a specific game
        
        Args:
            game_id: BGG game ID
            retry_delay: Seconds to wait between retries
            max_retries: Maximum number of retry attempts
            
        Returns:
            Tuple of (game_name, min_price_data, average_price)
        
        Raises:
            Exception: If API request fails or data processing fails
        """
        endpoint = f"{self.base_url}/thing"
        params = {
            "id": game_id,
            "type": "boardgame",
            "marketplace": 1 
        }
        
        response_data = None
        for attempt in range(max_retries):
            response = await api_request(url=endpoint, params=params)
            
            if response.status_code == 200:
                response_data = xmltodict.parse(response.content)
                # print(f"Debug: Response data for game ID {game_id}: {response_data}")  # Debug logging
                break
            elif response.status_code == 202:
                print(f"Request queued, retrying in {retry_delay} seconds...")
                await asyncio.sleep(retry_delay)
                continue
            else:
                response.raise_for_status()
        
        if not response_data:
            raise Exception(f"Failed to get response after {max_retries} attempts")
            
        try:
            # Extract game name
            if isinstance(response_data['items']['item']['name'], list):
                # If 'name' is a list, take the first element's '@value'
                game_name = response_data['items']['item']['name'][0]['@value']
            else:
                # If 'name' is not a list, directly access '@value'
                game_name = response_data['items']['item']['name']['@value']

            print(game_name)
            
            # Process marketplace listings
            min_value = {'link': '', 'value': float('inf')}
            price_list = []
            
            # Handle case where there might be single or multiple listings
            listings = response_data['items']['item'].get('marketplacelistings', {}).get('listing', [])
            if not isinstance(listings, list):
                listings = [listings]
                
            for listing in listings:
                if listing.get('price', {}).get('@currency') != 'EUR':
                    continue
                price = float(listing['price']['@value'])
                price_list.append(price)
                if price < min_value['value']:
                    min_value['link'] = listing.get('link', {}).get('@href', '')
                    min_value['value'] = price

            if not price_list:
                return (game_name, {'link': 'NA', 'value': float('inf')}, 0)

            avg_price = sum(price_list) / len(price_list)
            return (game_name, min_value, avg_price)
            
        except KeyError as e:
            raise Exception(f"Failed to process game data: {str(e)}")
        except ValueError as e:
            raise Exception(f"Failed to process pricing data: {str(e)}")
        except Exception as e:
            raise Exception(f"Unexpected error processing game data: {str(e)}")

if __name__ == "__main__":
    try:
        # Check if an event loop is already running
        try:
            loop = asyncio.get_running_loop()
        except RuntimeError:  # No event loop running
            loop = None

        if loop:
            # If running in an environment with an existing event loop (e.g., Jupyter notebook)
            import nest_asyncio
            nest_asyncio.apply()  # Allow nested event loops
            result = loop.run_until_complete(main())
        else:
            # If running as a standalone script
            result = asyncio.run(main())
        
        for i in result:
            print(i)
    except Exception as e:
        print(f"Error: {str(e)}")

Mare Mediterraneum
Tal der Könige
Samurai
El Caballero
Die Macher
Dragonmaster
Cathedral
Lords of Creation
Acquire
('Die Macher', {'link': 'https://boardgamegeek.com/market/product/1955338', 'value': 30.0}, 68.19566666666667)
('Dragonmaster', {'link': 'NA', 'value': inf}, 0)
('Samurai', {'link': 'https://boardgamegeek.com/market/product/3630338', 'value': 60.0}, 77.69153846153846)
('Tal der Könige', {'link': 'https://boardgamegeek.com/market/product/672651', 'value': 19.0}, 29.181818181818183)
('Acquire', {'link': 'https://boardgamegeek.com/market/product/3605513', 'value': 3.0}, 52.07411764705883)
('Mare Mediterraneum', {'link': 'https://boardgamegeek.com/market/product/650976', 'value': 175.0}, 175.0)
('Cathedral', {'link': 'https://boardgamegeek.com/market/product/2780856', 'value': 12.0}, 30.666666666666668)
('Lords of Creation', {'link': 'https://boardgamegeek.com/market/product/3040534', 'value': 9.0}, 20.567857142857143)
('El Caballero', {'link': 'https://boardgamegeek.com/marke

In [20]:
result

NameError: name 'result' is not defined

In [5]:
print(game_details['items']['item']['marketplacelistings'])

{'listing': [{'listdate': {'@value': 'Mon, 19 Nov 2018 19:35:29 +0000'}, 'price': {'@currency': 'GBP', '@value': '20.00'}, 'condition': {'@value': 'verygood'}, 'notes': {'@value': 'U.K. Shipping &#194;&#163;3, free collection from Brighton&#10;&#10;One token missing'}, 'link': {'@href': 'https://boardgamegeek.com/market/product/1673901', '@title': 'marketlisting'}}, {'listdate': {'@value': 'Mon, 15 Apr 2019 15:16:03 +0000'}, 'price': {'@currency': 'EUR', '@value': '20.00'}, 'condition': {'@value': 'likenew'}, 'notes': {'@value': 'opened but unplayed'}, 'link': {'@href': 'https://boardgamegeek.com/market/product/1817930', '@title': 'marketlisting'}}, {'listdate': {'@value': 'Sat, 25 May 2019 09:34:47 +0000'}, 'price': {'@currency': 'GBP', '@value': '7.00'}, 'condition': {'@value': 'verygood'}, 'notes': {'@value': 'Played only once'}, 'link': {'@href': 'https://boardgamegeek.com/market/product/1852584', '@title': 'marketlisting'}}, {'listdate': {'@value': 'Fri, 31 May 2019 23:13:12 +0000

In [7]:
for i in game_details['items']['item']['marketplacelistings']['listing']:
    print(i)

{'listdate': {'@value': 'Mon, 19 Nov 2018 19:35:29 +0000'}, 'price': {'@currency': 'GBP', '@value': '20.00'}, 'condition': {'@value': 'verygood'}, 'notes': {'@value': 'U.K. Shipping &#194;&#163;3, free collection from Brighton&#10;&#10;One token missing'}, 'link': {'@href': 'https://boardgamegeek.com/market/product/1673901', '@title': 'marketlisting'}}
{'listdate': {'@value': 'Mon, 15 Apr 2019 15:16:03 +0000'}, 'price': {'@currency': 'EUR', '@value': '20.00'}, 'condition': {'@value': 'likenew'}, 'notes': {'@value': 'opened but unplayed'}, 'link': {'@href': 'https://boardgamegeek.com/market/product/1817930', '@title': 'marketlisting'}}
{'listdate': {'@value': 'Sat, 25 May 2019 09:34:47 +0000'}, 'price': {'@currency': 'GBP', '@value': '7.00'}, 'condition': {'@value': 'verygood'}, 'notes': {'@value': 'Played only once'}, 'link': {'@href': 'https://boardgamegeek.com/market/product/1852584', '@title': 'marketlisting'}}
{'listdate': {'@value': 'Fri, 31 May 2019 23:13:12 +0000'}, 'price': {'@

In [32]:
for i in game_details['items']['item']:
    print(i)

@type
@id
thumbnail
image
name
description
yearpublished
minplayers
maxplayers
poll
poll-summary
playingtime
minplaytime
maxplaytime
minage
link
marketplacelistings


In [10]:
import requests
import xmltodict
import asyncio
from typing import Dict, Tuple, Union, Optional, List
import polars as pl

def prepare_data(response_data):

    game_info = response_data['items']['item']
    # Extract game name
    if isinstance(game_info['name'], list):
        # If 'name' is a list, take the first element's '@value'
        # Can have different names in different countries
        game_name = game_info['name'][0]['@value']
    else:
        # If 'name' is not a list, directly access '@value'
        game_name = game_info['name']['@value']
    game_description = game_info['description']
    game_publication_year = game_info['yearpublished']['@value']
    game_min_players = game_info['minplayers']['@value']
    game_max_players = game_info['maxplayers']['@value']
    # Polls parsing
    best_numplayers = None
    recommended_numplayers = None
    suggested_playerage = None
    language_dependence = None
    for poll in game_info['poll']:
        if poll['@name'] == 'suggested_numplayers':
            best_votes = 0
            recommended_votes = 0
            for result in poll['results']:
                for player_result in result.get('result', []):
                    if player_result['@value'] == 'Best' and int(player_result['@numvotes']) > best_votes:
                        best_numplayers = result['@numplayers']
                        best_votes = int(player_result['@numvotes'])
                    if player_result['@value'] == 'Recommended' and int(player_result['@numvotes']) > recommended_votes:
                        recommended_numplayers = result['@numplayers']
                        recommended_votes = int(player_result['@numvotes'])
        elif poll['@name'] == 'suggested_playerage':
            max_votes = 0
            for result in poll['results']['result']:
                votes = int(result['@numvotes'])
                if votes > max_votes:
                    suggested_playerage = result['@value']
                    max_votes = votes
        elif poll['@name'] == 'language_dependence':
            max_votes = 0
            for result in poll['results']['result']:
                votes = int(result['@numvotes'])
                if votes > max_votes:
                    language_dependence = {
                        'level': result['@level'],
                        'value': result['@value']
                    }
                    max_votes = votes
    # Categories and Mechanics
    game_categories = [
        link['@value'] for link in game_info['link'] 
        if link['@type'] == 'boardgamecategory'
    ]
    
    game_mechanics = [
        link['@value'] for link in game_info['link'] 
        if link['@type'] == 'boardgamemechanic'
    ]
    
    # Families
    game_families = [
        link['@value'] for link in game_info['link'] 
        if link['@type'] == 'boardgamefamily'
    ]
    
    # Designers
    game_designers = [
        link['@value'] for link in game_info['link'] 
        if link['@type'] == 'boardgamedesigner'
    ]
    
    # Artists
    game_artists = [
        link['@value'] for link in game_info['link'] 
        if link['@type'] == 'boardgameartist'
    ]
    
    # Publishers
    game_publishers = [
        link['@value'] for link in game_info['link'] 
        if link['@type'] == 'boardgamepublisher'
    ]
    
    # Playing Time
    game_playing_time = game_info['playingtime']['@value']
    game_min_playtime = game_info['minplaytime']['@value']
    game_max_playtime = game_info['maxplaytime']['@value']
    
    # Minimum Age
    game_min_age = game_info['minage']['@value']
    # Thumbnails and Images
    # game_thumbnail = game_info.get('thumbnail')
    # game_image = game_info.get('image')
    
    df = pl.DataFrame({
        "game_name": [game_name],
        "description": [game_description],
        "publication_year": [int(game_publication_year)],
        "min_players": [int(game_min_players)],
        "max_players": [int(game_max_players)],
        "best_num_players": [best_numplayers],
        "recommended_num_players": [recommended_numplayers],
        "suggested_play_age": [suggested_playerage],
        "categories": [game_categories],
        "mechanics": [game_mechanics],
        "families": [game_families],
        "designers": [game_designers],
        "artists": [game_artists],
        "publishers": [game_publishers],
        "playing_time": [int(game_playing_time)],
        "min_playtime": [int(game_min_playtime)],
        "max_playtime": [int(game_max_playtime)],
        "min_age": [int(game_min_age)],
        "language_dependence_level": [language_dependence['level'] if language_dependence else None],
        "language_dependence_description": [language_dependence['value'] if language_dependence else None]
    })
    return df

In [ ]:
import requests
import xmltodict
import asyncio
from typing import Dict, Tuple, Union, Optional, List
import polars as pl

def make_request(url: str, params: dict) -> requests.Response:
    response = requests.get(url=url, params=params)
    return response

async def api_request(url: str, params: dict) -> requests.Response:
    return await asyncio.to_thread(make_request, url, params)

async def main():
    client = BGGClient()
    tasks = [client.get_game_market_data(i) for i in range(1, 10)]
    result = await asyncio.gather(*tasks)
    return result

class BGGClient:
    def __init__(self, base_url: str = "https://boardgamegeek.com/xmlapi2"):
        self.base_url = base_url

    async def get_game_data(self, game_id: int, retry_delay: int = 5, max_retries: int = 3) -> Tuple[str, Dict[str, Union[str, float]], float]:
        """
        Fetches detailed market information for a specific game
        
        Args:
            game_id: BGG game ID
            retry_delay: Seconds to wait between retries
            max_retries: Maximum number of retry attempts
            
        Returns:
            ...
        
        Raises:
            Exception: If API request fails or data processing fails
        """
        endpoint = f"{self.base_url}/thing"
        params = {
            "id": game_id,
            "type": "boardgame",
            "marketplace": 1 
        }
        
        response_data = None
        for _ in range(max_retries):
            response = await api_request(url=endpoint, params=params)
            
            if response.status_code == 200:
                response_data = xmltodict.parse(response.content)
                # print(f"Debug: Response data for game ID {game_id}: {response_data}")  # Debug logging
                break
            elif response.status_code == 202:
                print(f"Request queued, retrying in {retry_delay} seconds...")
                await asyncio.sleep(retry_delay)
                continue
            else:
                response.raise_for_status()
        
        if not response_data:
            raise Exception(f"Failed to get response after {max_retries} attempts")

        pd_df = self.prepare_data(response_data)

        return pd_df
    
    def prepare_data(self, response_data):

        try:
            game_info = response_data['items']['item']
            # Extract game name
            if isinstance(game_info['name'], list):
                # If 'name' is a list, take the first element's '@value'
                # Can have different names in different countries
                game_name = game_info['name'][0]['@value']
            else:
                # If 'name' is not a list, directly access '@value'
                game_name = game_info['name']['@value']

            game_description = game_info['description']
            game_publication_year = game_info['yearpublished']['@value']
            game_min_players = game_info['minplayers']['@value']
            game_max_players = game_info['maxplayers']['@value']

            # Polls parsing
            best_numplayers = None
            recommended_numplayers = None
            suggested_playerage = None
            language_dependence = None

            for poll in game_info['poll']:
                if poll['@name'] == 'suggested_numplayers':
                    best_votes = 0
                    recommended_votes = 0
                    for result in poll['results']:
                        for player_result in result.get('result', []):
                            if player_result['@value'] == 'Best' and int(player_result['@numvotes']) > best_votes:
                                best_numplayers = result['@numplayers']
                                best_votes = int(player_result['@numvotes'])

                            if player_result['@value'] == 'Recommended' and int(player_result['@numvotes']) > recommended_votes:
                                recommended_numplayers = result['@numplayers']
                                recommended_votes = int(player_result['@numvotes'])

                elif poll['@name'] == 'suggested_playerage':
                    max_votes = 0
                    for result in poll['results']['result']:
                        votes = int(result['@numvotes'])
                        if votes > max_votes:
                            suggested_playerage = result['@value']
                            max_votes = votes

                elif poll['@name'] == 'language_dependence':
                    max_votes = 0
                    for result in poll['results']['result']:
                        votes = int(result['@numvotes'])
                        if votes > max_votes:
                            language_dependence = {
                                'level': result['@level'],
                                'value': result['@value']
                            }
                            max_votes = votes
    
            # Categories and Mechanics
            game_categories = [
                link['@value'] for link in game_info['link'] 
                if link['@type'] == 'boardgamecategory'
            ]
            
            game_mechanics = [
                link['@value'] for link in game_info['link'] 
                if link['@type'] == 'boardgamemechanic'
            ]
            
            # Families
            game_families = [
                link['@value'] for link in game_info['link'] 
                if link['@type'] == 'boardgamefamily'
            ]
            
            # Designers
            game_designers = [
                link['@value'] for link in game_info['link'] 
                if link['@type'] == 'boardgamedesigner'
            ]
            
            # Artists
            game_artists = [
                link['@value'] for link in game_info['link'] 
                if link['@type'] == 'boardgameartist'
            ]
            
            # Publishers
            game_publishers = [
                link['@value'] for link in game_info['link'] 
                if link['@type'] == 'boardgamepublisher'
            ]
            
            # Playing Time
            game_playing_time = game_info['playingtime']['@value']
            game_min_playtime = game_info['minplaytime']['@value']
            game_max_playtime = game_info['maxplaytime']['@value']
            
            # Minimum Age
            game_min_age = game_info['minage']['@value']
    
            # Thumbnails and Images
            # game_thumbnail = game_info.get('thumbnail')
            # game_image = game_info.get('image')
            
            df = pl.DataFrame({
                "game_name": [game_name],
                "description": [game_description],
                "publication_year": [int(game_publication_year)],
                "min_players": [int(game_min_players)],
                "max_players": [int(game_max_players)],
                "best_num_players": [int(best_numplayers)],
                "recommended_num_players": [int(recommended_numplayers)],
                "suggested_play_age": [int(suggested_playerage)],
                "categories": [game_categories],
                "mechanics": [game_mechanics],
                "families": [game_families],
                "designers": [game_designers],
                "artists": [game_artists],
                "publishers": [game_publishers],
                "playing_time": [int(game_playing_time)],
                "min_playtime": [int(game_min_playtime)],
                "max_playtime": [int(game_max_playtime)],
                "min_age": [int(game_min_age)],
                "language_dependence_level": [int(language_dependence['level']) if language_dependence else None],
                "language_dependence_description": [language_dependence['value'] if language_dependence else None]
            })

            return df 
        
        except KeyError as e:
            raise Exception(f"Failed to process game data: {str(e)}")
        except ValueError as e:
            raise Exception(f"Failed to process pricing data: {str(e)}")
        except Exception as e:
            raise Exception(f"Unexpected error processing game data: {str(e)}")
